In [1]:
import datetime
import os
import psycopg2
from pyspark.sql import (
    SparkSession,
    functions
)

In [2]:
postgres_schema="public"
postgres_table="aggregated_api_calls"

In [3]:

conn = psycopg2.connect(
    dbname=os.environ["POSTGRES_DB"],
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"],
    port=os.environ["POSTGRES_PORT"]
)

cur = conn.cursor()
cur.execute(f"SELECT MIN(created_date) FROM {postgres_schema}.{postgres_table}")
min_created_date = cur.fetchone()[0]  # Fetch the result and get the first column

cur.close()
conn.close()

print("Earliest created_date:", min_created_date)


Earliest created_date: 2025-06-15


In [5]:
# Start Spark session
spark = SparkSession.builder \
    .appName("ReadFromRDS") \
    .config("spark.jars", "/home/ollyhill/data-testing/mysql-connector-j-8.4.0.jar,/home/ollyhill/data-testing/postgresql-42.7.7.jar") \
    .getOrCreate()


# JDBC connection properties
jdbc_url = (
    "jdbc:mysql://" +
    os.environ["PORTAL_RDS"] +
    ":3306/" +
    os.environ["PORTAL_DB"]
)
connection_properties = {
    "user": os.environ['PORTAL_USER'],
    "password": os.environ['PORTAL_PASSWORD'],
    "driver": "com.mysql.cj.jdbc.Driver"
}

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 10:10:48 WARN Utils: Your hostname, OH-LAPTOP, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/24 10:10:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/24 10:10:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/24 10:10:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
end_index = 70028363
while end_index > 1:
    start_index = max(end_index - 100000, 1)
    query = f"""(
        SELECT id, user_id, device_id, created_at
        FROM api_calls
        WHERE id BETWEEN {start_index} AND {end_index}
    ) AS filtered"""

    df = spark.read.jdbc(
        url=jdbc_url,
        table=query,
        properties=connection_properties
    )
    min_ts = df.select(functions.min("created_at")).collect()[0][0]
    assert min_ts.date() < min_created_date - datetime.timedelta(days=1)
    min_created_date = min_ts.date() + datetime.timedelta(days=1)
    df = df \
        .filter(
            (functions.col("user_id").isNotNull()) &
            (functions.col("device_id").isNotNull()) &
            (functions.col("created_at").isNotNull()) &
            (functions.col("created_at") >= functions.lit(min_created_date.isoformat()))
        )

    df = df \
        .withColumn("created_date", functions.to_date("created_at"))  \
        .dropDuplicates(["user_id", "device_id", "created_date"])  \
        .select(["user_id", "device_id", "created_date"])
    end_index = start_index

datetime.date(2025, 6, 14)